In [6]:
import pandas as pd
import sqlite3

# Load the dataset
df = pd.read_csv("/Users/shivalimuthukumar/Desktop/Uber Request Data.csv")

# Connect to a temporary in-memory SQL database
conn = sqlite3.connect(":memory:")

# Load DataFrame into SQL table
df.to_sql("uber_requests", conn, index=False, if_exists='replace')


6745

In [8]:
print("Part 1: SQL Analysis\n")
# 1. Total number of requests by Status
query1 = """
SELECT Status, COUNT(*) AS Total_Requests
FROM uber_requests
GROUP BY Status;
"""
print("1. Total Requests by Status:")
print(pd.read_sql_query(query1, conn))

# 2. Hourly demand vs. supply
query2 = """
SELECT strftime('%H', "Request timestamp") AS Hour,
       SUM(CASE WHEN Status='Trip Completed' THEN 1 ELSE 0 END) AS Supply,
       SUM(CASE WHEN Status!='Trip Completed' THEN 1 ELSE 0 END) AS Demand_Gap
FROM uber_requests
GROUP BY Hour
ORDER BY Hour;
"""
print("\n2. Hourly Supply vs Demand Gap:")
print(pd.read_sql_query(query2, conn))

# 3. Requests by Pickup Point
query3 = """
SELECT "Pickup point",
       SUM(CASE WHEN Status='Trip Completed' THEN 1 ELSE 0 END) AS Completed,
       SUM(CASE WHEN Status='Cancelled' THEN 1 ELSE 0 END) AS Cancelled,
       SUM(CASE WHEN Status='No Cars Available' THEN 1 ELSE 0 END) AS No_Cars
FROM uber_requests
GROUP BY "Pickup point";
"""
print("\n3. Requests by Pickup Point:")
print(pd.read_sql_query(query3, conn))

# 4. Supply–Demand gap percentage
query4 = """
SELECT "Pickup point",
       COUNT(*) AS Total_Requests,
       SUM(CASE WHEN Status='Trip Completed' THEN 1 ELSE 0 END) AS Completed,
       ROUND(100.0 * SUM(CASE WHEN Status!='Trip Completed' THEN 1 ELSE 0 END) / COUNT(*), 2) AS Gap_Percentage
FROM uber_requests
GROUP BY "Pickup point";
"""
print("\n4. Supply-Demand Gap Percentage:")
print(pd.read_sql_query(query4, conn))


Part 1: SQL Analysis

1. Total Requests by Status:
              Status  Total_Requests
0          Cancelled            1264
1  No Cars Available            2650
2     Trip Completed            2831

2. Hourly Supply vs Demand Gap:
   Hour  Supply  Demand_Gap
0  None    2831        3914

3. Requests by Pickup Point:
  Pickup point  Completed  Cancelled  No_Cars
0      Airport       1327        198     1713
1         City       1504       1066      937

4. Supply-Demand Gap Percentage:
  Pickup point  Total_Requests  Completed  Gap_Percentage
0      Airport            3238       1327           59.02
1         City            3507       1504           57.11
